# Generating the EC coefficient

1. Import datasets using **ProjectZero modules**
2. Create all possible **combinations** or define options
3. Caluclate the embodied carbon /m2 for each option
4. Merge combinations data with corresponding **model data**


In [138]:
import pandas as pd
import ydata_profiling
import numpy as np
import random
from scripts.data import ProjectZero as pz

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---

# Creating A Dictionary

### 1. Importing Datasets: One touch templates, Model data, 

In [175]:
data = pz().get_data().copy()

df_model = data['hz_model'].drop(columns=['ID','Colour'])
df_options = data['sus_options']

### 2 .Defining the Options

**Define option 01** 

In [122]:
option_1 = df_options[df_options.building_type == 1]
option_1.head()

,building_type,building_use,basement_count,building_height,building_width,building_depth,element_type,building_element,value,unit
0,1,Office,0,25,20,20,Foundation,"Footing foundations for hard soils (sand, grav...",9.0,KgCO2e/m2
1,1,Office,0,25,20,20,Foundation,Steel core piling foundation for soft soils (15m),44.0,KgCO2e/m2
2,1,Office,0,25,20,20,Foundation,Rammed concrete piling foundation for hard soi...,12.0,KgCO2e/m2
3,1,Office,0,25,20,20,Ground slab,Concrete ground slab assembly incl. insulation,22.0,KgCO2e/m2
4,1,Office,0,25,20,20,Floor Slab,In-situ concrete slab assembly,82.0,KgCO2e/m2


**Define config 01**

In [106]:
key_names = list(option_1['element_type'].unique())
building_elements = [list(option_1['building_element'][option_1.element_type ==  key]) for key in key_names]

**Make dictionary of types {element_type: [building elements] }**

In [128]:
element_collection = {}

for key,element in zip(key_names, building_elements):
    element_collection[key] = element

print(element_collection)

{'Foundation': ['Footing foundations for hard soils (sand, gravel, silt or clay) per GFA, Includes: point and strip footings', 'Steel core piling foundation for soft soils (15m)', 'Rammed concrete piling foundation for hard soils (15m)'], 'Ground slab': ['Concrete ground slab assembly incl. insulation'], 'Floor Slab': ['In-situ concrete slab assembly', 'Concrete cast-in-situ floor slab assembly (150mm)', 'Metal-concrete composite decking (150mm)', 'Hollow-core slab floor assembly, incl. mineral wool acoustic slabs', 'CLT floor slab assembly (200mm)'], 'Columns': ['Reinforced wall, for concrete building', 'Precast concrete column', 'In-situ concrete column', 'Timber column - for timber frame buildings', 'Steel column'], 'Shear Wall': ['Reinforced concrete shear wall'], 'Beams': ['Precast concrete beam', 'In-situ concrete beam', 'Timber beam - for timber frame buildings', 'Steel beam'], 'Stairs': ['Concrete assembly for stairs and elevator shafts per one metre height', 'CLT assembly for 

### 3. Selecting random configuration from collection 

In [120]:
element_collection.keys()
selector = random.randint(0,len(key_names)-1)

list(element_collection.keys())[selector]

'Beams'

### 4. Adding interactive Selection

In [137]:
import ipywidgets as widgets
from IPython.display import display

# Create the first dropdown for selecting keys
dropdown_keys = widgets.Dropdown(
    options=list(element_collection.keys()),
    value=list(element_collection.keys())[0],
    description='Select element type:'
)

# Create the second dropdown for selecting options from the chosen key
dropdown_options = widgets.Dropdown(
    options=element_collection[dropdown_keys.value],
    value=element_collection[dropdown_keys.value][0],
    description='Select building element:'
)

# Display the dropdowns
display(dropdown_keys, dropdown_options)

# Function to update the options in the second dropdown based on the first dropdown's value
def update_options(change):
    selected_key = change['new']
    dropdown_options.options = element_collection[selected_key]
    dropdown_options.value = element_collection[selected_key][0]

# Attach the update function to the first dropdown's value change event
dropdown_keys.observe(update_options, names='value')

# Function to handle the second dropdown value changes
def on_dropdown_options_change(change):
    selected_option = change['new']
    print(f'Selected option: {selected_option}')

# Attach the function to the second dropdown's value change event
dropdown_options.observe(on_dropdown_options_change, names='value')


Dropdown(description='Select element type:', options=('Foundation', 'Ground slab', 'Floor Slab', 'Columns', 'S…

Dropdown(description='Select building element:', options=('Footing foundations for hard soils (sand, gravel, s…

Selected option: Reinforced concrete shear wall
Selected option: Concrete assembly for stairs and elevator shafts per one metre height
Selected option: CLT assembly for stairs and elevator shaft


## 

#  Create A Dataframe

In [156]:
option_1 = df_options[df_options.building_type == 1]
option_2 = df_options[df_options.building_type == 2]

option_1.head(3)

,building_type,building_use,basement_count,building_height,building_width,building_depth,element_type,building_element,value,unit
0,1,Office,0,25,20,20,Foundation,"Footing foundations for hard soils (sand, grav...",9.0,KgCO2e/m2
1,1,Office,0,25,20,20,Foundation,Steel core piling foundation for soft soils (15m),44.0,KgCO2e/m2
2,1,Office,0,25,20,20,Foundation,Rammed concrete piling foundation for hard soi...,12.0,KgCO2e/m2


### Test Configuration: First Elements when grouped by `element_type`

In [154]:
config_1 = option_1.groupby('element_type', as_index=False).agg({'building_type': 'first', 'building_element': 'first', 'value': 'first'})
config_1

,element_type,building_type,building_element,value
0,Beams,1,Precast concrete beam,39.0
1,Columns,1,"Reinforced wall, for concrete building",26.0
2,External Walls,1,"Timber frame external wall assembly, incl. min...",9.0
3,Floor Slab,1,In-situ concrete slab assembly,82.0
4,Foundation,1,"Footing foundations for hard soils (sand, grav...",9.0
5,Ground slab,1,Concrete ground slab assembly incl. insulation,22.0
6,Shear Wall,1,Reinforced concrete shear wall,4.0
7,Stairs,1,Concrete assembly for stairs and elevator shaf...,8.0


In [157]:
config_2 = option_2.groupby('element_type', as_index=False).agg({'building_type': 'first', 'building_element': 'first', 'value': 'first'})
config_2

,element_type,building_type,building_element,value
0,Beams,2,Precast concrete beam,65.0
1,Columns,2,Precast concrete column,20.0
2,Connecting parts,2,Steel connecting parts,1.0
3,Diagonal wind bracings,2,"Steel wind bracing, X-shaped, per m2 external ...",3.0
4,Floor Slab,2,In-situ concrete slab assembly,93.0
5,Foundation,2,Steel core piling foundation for soft soils (2...,92.0
6,Ground slab,2,Concrete ground slab assembly incl. insulation,31.0
7,Secondary beams,2,Secondary steel beam,15.0
8,Stairs,2,Concrete assembly for stairs and elevator shaf...,12.0


### Calculating the coefficient for each table

In [161]:
coefficient_1 = config_1['value'].sum()
coefficient_2 = config_2['value'].sum()

coefficient_2

332.0

### Applying to table

In [176]:
df_model['building_type'] = 1
df_model['embodied_carbon'] = df_model['Area'] * coefficient_1

df_model

,Typology,Area,Plot,Building,building_type,embodied_carbon
0,Commercial,1696.329992,YH090703-05,,1,337569.668408
1,Commercial,2896.125039,YH090703-04,Tower 03,1,576328.882761
2,Commercial,2056.249041,YH090703-04,Tower 03,1,409193.559159
3,Commercial,2056.249041,YH090703-04,Tower 03,1,409193.559159
4,Commercial,1696.329992,YH090703-05,,1,337569.668408
...,...,...,...,...,...,...
571,Retail,79.068583,YH090703-05,,1,15734.648017
572,Retail,121.818583,YH090703-05,,1,24241.898017
573,Retail,191.118583,YH090703-05,,1,38032.598017
574,Retail,79.068583,YH090703-05,,1,15734.648017
